In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
from sklearn import preprocessing
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (16, 9)

In [ ]:
pd.set_option('display.max_columns',150)
pd.set_option('display.max_rows',90)

In [ ]:
dataset=pd.read_csv("drive/MyDrive/Colab Notebooks/TFM/Recaidas_colon.csv")

In [ ]:
dataset.describe()

,NHC,Telfono,AF CCR primer grado,MT CEA(marcador tumoral CEA),"Ca 19.9( marcador tumoral, se suele estudiar la cantidad de este maracdor en sangre",Tumor estenosante(lesion estenosante del colon identifcable en uan exploración física),PT PeriOP,Hb Preqx,Leu PreQx,PCR preQx,PCR 3D,PCR 5D,Fecha alta,AJCC stage(sistema de estadificación),pT stage,pN stage,Diferenciación,Diametro maximo,Espesor,Porcentaje circunferencia,GG aislados,GG +,Inestabilida microsatelites,QT adyuvante( como terapia adyuvante se ha utilizado la quimioterapia,Tipo QT (tipo de qiomioterapia),Recaída,"Exitus( si han muerto o no, pregunatr porque hay casos en lsoq ue pone 2)"
count,1.110000e+02,8.500000e+01,0.0,0.0,0.0,0.0,94.000000,0.0,0.0,0.0,83.000000,78.000000,0.0,0.0,0.0,0.0,0.0,105.000000,45.000000,102.000000,108.00000,108.000000,0.0,0.0,0.0,110.000000,111.000000
mean,4.382908e+05,8.174828e+08,NaN,NaN,NaN,NaN,6.703191,NaN,NaN,NaN,138.021687,101.138462,NaN,NaN,NaN,NaN,NaN,40.400000,11.422222,64.656863,17.12037,0.925926,NaN,NaN,NaN,0.127273,0.171171
std,5.537879e+05,1.495043e+08,NaN,NaN,NaN,NaN,0.646155,NaN,NaN,NaN,111.711610,84.833010,NaN,NaN,NaN,NaN,NaN,19.614849,5.626596,31.103944,7.33538,2.335039,NaN,NaN,NaN,0.334804,0.444642
min,7.594000e+03,9.131302e+07,NaN,NaN,NaN,NaN,4.500000,NaN,NaN,NaN,11.400000,10.000000,NaN,NaN,NaN,NaN,NaN,8.000000,4.000000,10.000000,2.00000,0.000000,NaN,NaN,NaN,0.000000,0.000000
25%,1.878050e+05,6.599059e+08,NaN,NaN,NaN,NaN,6.400000,NaN,NaN,NaN,50.300000,37.000000,NaN,NaN,NaN,NaN,NaN,30.000000,7.000000,36.250000,12.00000,0.000000,NaN,NaN,NaN,0.000000,0.000000
50%,3.056740e+05,9.131323e+08,NaN,NaN,NaN,NaN,6.800000,NaN,NaN,NaN,109.000000,80.000000,NaN,NaN,NaN,NaN,NaN,40.000000,10.000000,70.000000,17.00000,0.000000,NaN,NaN,NaN,0.000000,0.000000
75%,6.132430e+05,9.160164e+08,NaN,NaN,NaN,NaN,7.100000,NaN,NaN,NaN,192.500000,132.500000,NaN,NaN,NaN,NaN,NaN,50.000000,18.000000,100.000000,20.25000,1.000000,NaN,NaN,NaN,0.000000,0.000000
max,5.519850e+06,9.185945e+08,NaN,NaN,NaN,NaN,7.800000,NaN,NaN,NaN,504.000000,379.000000,NaN,NaN,NaN,NaN,NaN,105.000000,20.000000,100.000000,40.00000,12.000000,NaN,NaN,NaN,1.000000,2.000000


In [ ]:
dataset.head(20)

In [ ]:
dataset.isna()

In [ ]:
dataset.columns.values

In [ ]:
dataset.shape

(124, 81)

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().values.any()


True

In [ ]:
dataset.isnull()


In [ ]:
dataset.isnull().any()


In [ ]:
nulls = dataset.isnull().sum()

In [ ]:
index = nulls.index

In [ ]:
df_na = pd.DataFrame(nulls, columns= ['Nº_na_values'])
df_na['index']=df_na.index

In [ ]:
df_na.head()

In [ ]:
nulls.dtype

dtype('int64')

In [ ]:
nulls.index

In [ ]:
df_na.shape

(81, 2)

In [ ]:
df_na.head()

In [ ]:
#df_na = df_na.drop(df_na.loc[df_na['Nº_na_values']==124].index)

In [ ]:
# Get names of indexes for which column Stock has value No
indexNames = df_na[ df_na['Nº_na_values'] == 124 ].index
# Delete these row indexes from dataFrame
df_na.drop(indexNames , inplace=True)
print(df_na)
#otra forma de realizar lo mismo

In [ ]:
#hemos eliminado 15 columnas ya que estaban en blanco completamente

In [ ]:
df_na.shape

(66, 2)

In [ ]:
df_na.head(66)

In [ ]:
dataset_new = dataset.drop(indexNames, axis = 1)

In [ ]:
dataset_new.tail(30)

In [ ]:
dataset_new = dataset_new.dropna(axis = 0, subset=['Paciente'])

In [ ]:
dataset_new.shape

(111, 66)

In [ ]:
dataset_new.set_index('Paciente',inplace = True)


In [ ]:
dataset_new.head()

In [ ]:
lista = ['Telfono', 'Fecha cirugia', 'Fecha nacimiento', 'Fecha Recaída', 'Fecha ultimo seg']

In [ ]:
dataset_new = dataset_new.drop(lista, axis = 1)

In [ ]:
dataset_new.shape

(111, 60)

In [ ]:
df = dataset_new

In [ ]:
df.head(111)

In [ ]:
df = df.drop(['fistula'],axis = 1)

In [ ]:
from pyspark.ml.feature import (VectorAssembler,StringIndexer,OneHotEncoder)


In [ ]:
nan = df.isnull().sum()

nan.head(111)

In [ ]:
nan = pd.DataFrame(nan)

In [ ]:
nan.to_excel('NaN_values_cancer.xlsx')

In [ ]:
index = StringIndexer(inputCol=column, outputCol =column+'index' ) for column in list()
model = index.fit(df)
df = model.transfrom(df)

SyntaxError: ignored